In [1]:
import torch
import torch.nn.functional as F
import torchvision
from torch.autograd import Variable
import numpy as np
from matplotlib import pyplot as plt
import time
import cv2 as cv

In [2]:
trainds = torchvision.datasets.CIFAR10(
    root = "./data",
    train = True,
    transform = torchvision.transforms.ToTensor(),  
    download = True
)

testds = torchvision.datasets.CIFAR10(
    root = "./data",
    train = False,
    transform = torchvision.transforms.ToTensor(), 
    download = True
)

batch_size = 100

trainldr = torch.utils.data.DataLoader(
    dataset = trainds,
    batch_size = batch_size,
    shuffle = True
)

testldr = torch.utils.data.DataLoader(
    dataset = testds,
    batch_size = batch_size
)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
img_hwc = np.array(trainds[0][0] * 255).astype(np.uint8).transpose((1, 2, 0))
img_hwc = cv.resize(img_hwc, (100, 100), interpolation = cv.INTER_LINEAR)

#plt.imshow(img_hwc)

In [4]:
class cnn(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.mxpul = torch.nn.MaxPool2d((2, 2), stride = 2)

        self.convlay0 = torch.nn.Conv2d(3, 64, 3, stride = 1, padding = 'same')
        self.convlay1 = torch.nn.Conv2d(64, 64, 3, stride = 1, padding = 'same')
        self.convlay2 = torch.nn.Conv2d(64, 128, 3, stride = 1, padding = 'same')
        self.convlay3 = torch.nn.Conv2d(128, 128, 3, stride = 1, padding = 'same')
        self.convlay4 = torch.nn.Conv2d(128, 256, 3, stride = 1, padding = 'same')
        self.convlay5 = torch.nn.Conv2d(256, 256, 3, stride = 1, padding = 'same')
        self.convlay6 = torch.nn.Conv2d(256, 512, 3, stride = 1, padding = 'same')
        self.convlay7 = torch.nn.Conv2d(512, 512, 3, stride = 1, padding = 'same')
        self.convlay8 = torch.nn.Conv2d(512, 512, 3, stride = 1, padding = 'same')

        self.linlay0 = torch.nn.Linear(512 * 4, 1024)
        self.linlay1 = torch.nn.Linear(1024, 1024)
        self.linlay2 = torch.nn.Linear(1024, 10)
    
    def forward(self, x):

        out = F.relu(self.convlay0(x))
        out = self.mxpul(F.relu(self.convlay1(out)))
        out = F.relu(self.convlay2(out))
        out = self.mxpul(F.relu(self.convlay3(out)))
        out = F.relu(self.convlay4(out))
        out = self.mxpul(F.relu(self.convlay5(out)))
        out = F.relu(self.convlay6(out))
        out = F.relu(self.convlay7(out))
        out = self.mxpul(F.relu(self.convlay8(out)))
        
        out = out.view(-1, 512 * 2 * 2)
        out = F.relu(self.linlay0(out))
        out = F.relu(self.linlay1(out))
        out = self.linlay2(out)
        
        return F.softmax(out, dim=1)
#class cnn(torch.nn.Module):
#    def __init__(self):
#        super().__init__()
#        
#        self.mxpul = torch.nn.MaxPool2d((2, 2), stride = 1)
#
#        self.convlay0 = torch.nn.Conv2d(3, 64, 3, stride = 1)
#        self.convlay1 = torch.nn.Conv2d(64, 128, 3, stride = 2)
#        self.convlay2 = torch.nn.Conv2d(128, 128, 3, stride = 3)
#
#        self.linlay0 = torch.nn.Linear(1152, 300)
#        self.linlay1 = torch.nn.Linear(300, 10)
#    
#    def forward(self, x):
#
#        out = self.mxpul(F.relu(self.convlay0(x)))
#        out = self.mxpul(F.relu(self.convlay1(out)))
#        out = self.mxpul(F.relu(self.convlay2(out)))
#        out = out.view(-1, 128 * 3 * 3)
#        out = F.relu(self.linlay0(out))
#        out = self.linlay1(out)
#        
#        return F.softmax(out, dim=1)

In [6]:
net = cnn()
print(net)

Is CUDA enabled? False
cnn(
  (mxpul): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  (convlay0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (convlay1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (convlay2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (convlay3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (convlay4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (convlay5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (convlay6): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (convlay7): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (convlay8): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (linlay0): Linear(in_features=2048, out_features=1024, bias=True)
  (linlay1): Linear(in_features=1024, out_features=1024, bias=True)
  (linlay2): Linear(in_features

In [7]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

In [8]:
criterion = torch.nn.CrossEntropyLoss()
print(len(trainldr.dataset))

50000


In [9]:
epochs = 20

for epoch in range(epochs):
    start_time = time.time()
    mean_loss = 0
    for data, labels in trainldr:
        data, labels = Variable(data), Variable(labels)
        
        optimizer.zero_grad()
        net_out = net(data)
        loss = criterion(net_out, labels)
        mean_loss += loss.data
        loss.backward()
        optimizer.step()
    print("number epoch: {} \t epoch execution time: {} \t  mean_loss: {}".format(
        epoch + 1, time.time() - start_time, mean_loss / len(trainldr.dataset)))

number epoch: 1 	 epoch execution time: 244.19639039039612 	  mean_loss: 0.023025887086987495
number epoch: 2 	 epoch execution time: 256.7474799156189 	  mean_loss: 0.02302587404847145
number epoch: 3 	 epoch execution time: 257.7462751865387 	  mean_loss: 0.023025866597890854
number epoch: 4 	 epoch execution time: 261.7996048927307 	  mean_loss: 0.0230258759111166
number epoch: 5 	 epoch execution time: 322.5918209552765 	  mean_loss: 0.023025883361697197
number epoch: 6 	 epoch execution time: 286.6722548007965 	  mean_loss: 0.0230258796364069
number epoch: 7 	 epoch execution time: 238.62143206596375 	  mean_loss: 0.023025866597890854
number epoch: 8 	 epoch execution time: 239.38139033317566 	  mean_loss: 0.0230258721858263
number epoch: 9 	 epoch execution time: 237.05193090438843 	  mean_loss: 0.0230258721858263
number epoch: 10 	 epoch execution time: 239.52543377876282 	  mean_loss: 0.02302587404847145
number epoch: 11 	 epoch execution time: 239.3646056652069 	  mean_loss: 0

In [15]:
test_loss = 0
correct = 0
for data, labels in testldr:
    data, labels = Variable(data), Variable(labels)
    net_out = net(data)
    test_loss += criterion(net_out, labels).data
    for ind, t in enumerate(net_out):
        mxind1, mxind2 = 0, 1
        for ind2, x in enumerate(t):
            if t[ind2] > t[mxind1]:
                mxind2 = mxind1
                mxind1 = ind2
            elif ind2 != mxind1 and t[ind2] > t[mxind2]:
                mxind2 = ind2
        if labels[ind] == mxind1 or labels[ind] == mxind2:
            correct += 1
   

test_loss /= len(testldr.dataset)
print('nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
       test_loss, correct, len(testldr.dataset),
       100. * correct / len(testldr.dataset)))

nTest set: Average loss: 0.0230, Accuracy: 2000/10000 (20%)
